# In this Notebook I convert the data of the scans I created in the Scans_Data_Processing file to be suitable for training

In [ ]:
#Data Proccesing
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
Project_Path='Local Path'

I load the Scans_Data2 file I created into the Scans_Data_Processing notebook

In [ ]:
Scans_Data=pd.read_csv(Project_Path+ '/Data/Scan_Data2.csv',low_memory=False,sep=',',index_col=0)
Scans_Data

I am converting the format of the "Date" column from "Year/Month/Day" to "Month/Day/Year" so that it has the appropriate format so that I can then convert it to pandas datetime

In [ ]:
Date_List=[]
Scans_List=Scans_Data.values.tolist()
for i in tqdm(range(0,len(Scans_List))):
    Date = Scans_List[i][0]
    y,m,d = Date.split('-')
    D=m + '/' + d + '/' +y
    Date_List.append(D)
Scans_Data=Scans_Data.drop(['Date'], axis=1)
Scans_Data.insert(1, "Date", Date_List, True)
Scans_Data

Μετατρέπω τις στήλες 'Date' και 'Time' σε pandas datetime ώστε να μετατρέψω την ώρα σε time-int (Seconds) 

In [ ]:
Scans_Data['Date_Time'] = Scans_Data['Date'].map(str)+ ' ' + Scans_Data['Time'].map(str)
Scans_Data['Date_Time'] = pd.to_datetime(Scans_Data['Date_Time'])
Scans_Data['Time_Int'] = (Scans_Data['Date_Time']-Scans_Data['Date_Time'].dt.normalize()).dt.total_seconds()
Scans_Data=Scans_Data.drop(['Time'], axis=1)
Scans_Data=Scans_Data.drop(['Date_Time'], axis=1)
Scans_Data

Knowing from the previous notebook what are the times when I have recorded scans (6:00, 7:00,..., 19:00, 20:00) I give their imprint in seconds, for example (6:00 = 21600) and then I capture her with their center. eg (6:00 -> 6:30) and therefore (21600->23400). So I ended up with a list of 15 hours that will be used to sort the scans into time slots

In [ ]:
Time_Slots=[21600,25200,28800,32400,36000,39600,43200,46800,50400,54000,57600,61200,64800,68400,72000]
Time_SlotsCenter=[]
for i in range (0,len(Time_Slots)):
    Time_SlotsCenter.append(Time_Slots[i]+1800)
Time_Slots=Time_SlotsCenter
Time_Slots

According to the above list I classified each scan in the time-slot that was closest. That is, a scan that was done at 12:45 went to the time-slot '12:00-13:00' which is captured as 12:00. That is, I added the Time_Slot column. I also added the Time_Distance column which states how far in seconds the pre-wedding time of the scan is from the center of the timeslot we ranked the scan. I use Time_Distance to check for ranking errors.

In [ ]:
Scan_List=Scans_Data.values.tolist()
Real_Time_Slots=[]
Time_Distance=[]
for i in tqdm(range(0,len(Scan_List))):
    Real_Time=Scan_List[i][12]
    Distances=[]
    Slots=[]
    
    for j in range (0,len(Time_Slots)):
        Distances.append(abs(Time_Slots[j]-Real_Time))
        
    Slots=np.column_stack((Time_Slots, Distances))
    Slots = sorted(Slots, key=lambda x: x[1])
    Real_Time_Slots.append(Slots[0][0])
    Time_Distance.append(Slots[0][1])
Scans_Data.insert(13, "Time_Slot", Real_Time_Slots, True)   
Scans_Data.insert(14, "Time_Distance", Time_Distance, True)   
Scans_Data

I deleted the scans done during the lock down period. I explain in the report why I did this

In [ ]:
Scans_Data.drop(Scans_Data.index[Scans_Data['Lockdown'] == 1], inplace=True)
Scans_Data=Scans_Data.reset_index()
del Scans_Data["index"]
Scans_Data

I keep only the columns I need.

In [ ]:
Scans_Data2=Scans_Data['Slot_id'],Scans_Data['Date'],Scans_Data['Day Name'], Scans_Data['Time_Int'], Scans_Data['Time_Slot'],Scans_Data['Covid'],Scans_Data['Legal']
Headers=['Slot_id','Date','Day Name','Time_Int','Time_Slot','Covid','Legal']
Scans_Data2 = pd.concat(Scans_Data2, axis=1, keys=Headers)
Scans_Data2

I convert the Legal column to categorical to then do one-hot encoding. (This step is unnecessary)

In [ ]:
Scans_Data2['Legal'] = Scans_Data2['Legal'].replace(1, 'Legal')
Scans_Data2['Legal'] = Scans_Data2['Legal'].replace(0, 'Ilegal')
Scans_Data2

I do one hot encoding of the legal column, create 2 new columns through this process and delete the legal column. Then I save the dataframe to have this point as a check-point

In [ ]:
# creating initial dataframe
Legality_Types = ('Legal','Ilegal')
Legaldf = pd.DataFrame(Scans_Data2, columns=['Legal'])
# generate binary values using get_dummies
Legality = pd.get_dummies(Legaldf, columns=["Legal"], prefix=["Legality"] )
# merge with main df bridge_df on key values
Scans_Data2 = Scans_Data2.join(Legality)
Scans_Data2=Scans_Data2.drop(['Legal'], axis=1)
Scans_Data2.to_csv(Project_Path+ '/Data/Scan_Data_Reg_2.1.csv')
Scans_Data2

I convert some of the columns into a string that will be used as a key. I leave out the column specifying the actual time of each scan and the columns specifying the parking violation. The key contains information about the time domain, the time-slot. So samples with the same key are scans made on the same day in the same time-slot in the same sector

In [ ]:
Scans_Data2=pd.read_csv(Project_Path+ '/Data/Scan_Data_Reg_2.1.csv',low_memory=False,sep=',',index_col=0)
Scans_Data2['Key'] = Scans_Data2['Slot_id'].map(str)+ '+' +Scans_Data2['Date'].map(str) + '+' + Scans_Data2['Day Name'].map(str)+ '+' + Scans_Data2['Time_Slot'].map(str)+ '+'+Scans_Data2['Covid'].map(str)
Scans_Data2=Scans_Data2['Key'],Scans_Data2['Time_Int'],Scans_Data2['Legality_Ilegal'],Scans_Data2['Legality_Legal']
Headers=['Key','Time_Int','Legality_Ilegal','Legality_Legal']
Scans_Data2 = pd.concat(Scans_Data2, axis=1, keys=Headers)
Scans_Data2


I group by and for each different key I calculate the average time of the scans, the scans that showed illegality and the scans that showed legitimacy. Essentially with this process I collect the scans to create the controls, a control consists of scans done on the same day in the same time-slot in the same sector

In [ ]:
Time_Int = Scans_Data2.groupby(["Key"]).Time_Int.mean().reset_index()
ILegal_Sum = Scans_Data2.groupby(["Key"]).Legality_Ilegal.sum().reset_index()
Legal_Sum= Scans_Data2.groupby(["Key"]).Legality_Legal.sum().reset_index()
Legal_Sum=Legal_Sum['Key'],Time_Int['Time_Int'],ILegal_Sum['Legality_Ilegal'],Legal_Sum['Legality_Legal']
Headers=['Key','Time_Int','Legality_Ilegal','Legality_Legal']
Legal_illegal = pd.concat(Legal_Sum, axis=1, keys=Headers)
Legal_illegal

I calculate how many scans I got on each check and put it in the sum column then delete all the checks that resulted from 1 or two scans

In [ ]:
Legal_illegal['Sum']=Legal_illegal['Legality_Ilegal']+Legal_illegal['Legality_Legal']
Legal_illegal.drop(Legal_illegal[Legal_illegal.Sum <= 2].index, inplace=True)
Legal_illegal=Legal_illegal.reset_index()
del Legal_illegal["index"]
Legal_illegal

I calculate for each scan the violation rate with the formula -> Rate = Illegal Scans/Total Scans. I also explain in the report in more detail.

In [ ]:
Scan_List2=Legal_illegal.values.tolist()
Ilegality_Rate=[]
for i in range (0,len(Scan_List2)):
    if Scan_List2[i][2]==0:
        Ilegality_Rate.append(0)
    else:
        Ilegality_Rate.append(Scan_List2[i][2]/(Scan_List2[i][2]+Scan_List2[i][3]))
Legal_illegal.insert(3, "Ilegality_Rate", Ilegality_Rate, True)
Legal_illegal=Legal_illegal.drop(['Legality_Ilegal'], axis=1)
Legal_illegal=Legal_illegal.drop(['Legality_Legal'], axis=1)
Legal_illegal

Again I split the string key to create the original columns that made it up (This group by process is also done in a simpler way without having to make the string columns and then make the string columns but when I did it I didn't know which is also done in a faster way with pandas)

In [ ]:
Scan_List2=Legal_illegal.values.tolist()
Slot_id=[]
Date=[]
Day_Name=[]
Slot_Timeint=[]
Covid=[]
for i in tqdm(range(0,len(Scan_List2))):
    Slot_id_Value,Date_Value,Day_Name_Value,Slot_Timeint_Value,Covid_Value=Scan_List2[i][0].split('+')
    Slot_id.append(Slot_id_Value)
    Date.append(Date_Value)
    Day_Name.append(Day_Name_Value)
    Slot_Timeint.append(Slot_Timeint_Value)
    Covid.append(Covid_Value)
Legal_illegal.insert(1, "Slot_id", Slot_id, True)
Legal_illegal.insert(2, "Date", Date, True)
Legal_illegal.insert(3, "Day_Name",  Day_Name, True)
Legal_illegal.insert(4, "Slot_Timeint", Slot_Timeint, True)
Legal_illegal.insert(5, "Covid", Covid, True)
Legal_illegal=Legal_illegal.drop(['Key'], axis=1)
Legal_illegal

I am converting the formats of some columns so that they are not strings

In [ ]:
Legal_illegal['Slot_Timeint'] = Legal_illegal['Slot_Timeint'].astype(float)
Legal_illegal['Time_Int'] = Legal_illegal['Time_Int'].astype(float)
Legal_illegal['Slot_Timeint'] = Legal_illegal['Slot_Timeint'].astype(int)
Legal_illegal['Time_Int'] = Legal_illegal['Time_Int'].astype(int)
Legal_illegal

I load the file that describes the domains, and according to that I create a column in the data set that describes the capacity in places of the domain that each control refers to. (And this is done more easily with merge than pandas, but when I wrote this file I didn't know how to work with pandas to the point where I knew these functions were there ready-made)

In [ ]:
Column_Names=['Slot_id','Adress','Mean','Start','End','Capacity','Full_adress','Latitude','Longitude']

Parking_Slots2=pd.read_csv(Project_Path+ '/Data/Parkink_Slot_Proccesed.csv',sep=',', names=Column_Names)

Parking_Slots2=Parking_Slots2.values.tolist()
Scans_List2=Legal_illegal.values.tolist()
Capacity=[]
for i in tqdm(range(0,len(Scans_List2))):
    for j in range (0,len(Parking_Slots2)):
        if int(Parking_Slots2[j][0])==int(Scans_List2[i][0]):
            c=int(Parking_Slots2[j][5])
            Capacity.append(c) 



Legal_illegal.insert(7, "Capacity",Capacity, True)
Legal_illegal

According to the number of scans made in each domain and according to the capacity of the domain, I create the column Attention which describes to what percentage the domain was checked.

In [ ]:
Scan_List2=Legal_illegal.values.tolist()
Attention=[]
for i in range (0,len(Scan_List2)):
    if Scan_List2[i][8]>Scan_List2[i][7]:
        Attention.append(1)
    else:
        Attention.append(Scan_List2[i][8]/(Scan_List2[i][7]))
Legal_illegal.insert(7, "Attention", Attention, True)

Legal_illegal



I delete audits where less than 20% of each domain was audited

In [ ]:
Legal_illegal.drop(Legal_illegal[Legal_illegal.Attention <= 0.2].index, inplace=True)
Legal_illegal.to_csv(Project_Path+ '/Data/Scan_Data_For_Analysis.csv')
Legal_illegal=Legal_illegal.drop(['Attention'], axis=1)
Legal_illegal=Legal_illegal.reset_index()
del Legal_illegal["index"]
Legal_illegal

I load the file that describes the public holidays and create a column based on it that describes in a binary way whether the check was made on a public holiday or not.

In [ ]:
Holidays= pd.read_csv(Project_Path+ '/Data/Holidays.txt', header = None,names=['Holidays'])
Holidays_List=Holidays.values.tolist()
Scan_List2=Legal_illegal.values.tolist()
Holidays=[]
for i in tqdm(range(0,len(Scan_List2))):
    Boulean=False
    for j in range (0,len(Holidays_List)):
        if Scan_List2[i][1]==Holidays_List[j][0]:
            Boulean=True
    
    if Boulean:
        Holidays.append(1)
    else:
        Holidays.append(0)
        
Legal_illegal.insert(7, "Holidays", Holidays, True)
Legal_illegal

I parse the Date column into: days of the month, month, year

In [ ]:
Scan_List2=Legal_illegal.values.tolist()

Date=[]
Year=[]
Mounth=[]
Key_Date=[]

for i in tqdm(range(0,len(Scan_List2))):
    m,d,y=Scan_List2[i][1].split('/')
    Mounth.append(int(m))
    Date.append(int(d))
    Year.append(int(y))
    D=y + '-' + m + '-' +d
    Key_Date.append(D)

Legal_illegal.insert(1, "Key_Date", Key_Date, True)
Legal_illegal.insert(2, "Date_Of_Month", Date, True)
Legal_illegal.insert(3, "Year", Year, True)
Legal_illegal.insert(4, "Month", Mounth, True)
Legal_illegal=Legal_illegal.drop(['Date'], axis=1)
Legal_illegal

I save the Dataframe as a csv so that I have this point as a checkpoint

In [ ]:
Legal_illegal.to_csv(Project_Path+ '/Data/Scan_Data_Reg_2.2.csv')

In [ ]:
Legal_illegal=pd.read_csv(Project_Path+ '/Data/Scan_Data_Reg_2.2.csv',sep=',',index_col=0)
Legal_illegal

I convert the days of the week into numbers, i.e. the second became 1 the third became 2 etc etc so that I can format it afterwards

In [ ]:
List=Legal_illegal.values.tolist()
Week_Day=[]
Legal_illegal=Legal_illegal.drop(['Day_Name'], axis=1)
Day_Name_To_Day_Slot=[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],[1,2,3,4,5,6,7]]
for i in tqdm(range(0,len(List))):
    for j in range(0,7):
        if List[i][5]==Day_Name_To_Day_Slot[0][j]:
            Week_Day.append(Day_Name_To_Day_Slot[1][j])
   
Legal_illegal.insert(2, "Week_Day", Week_Day, True)
Legal_illegal

I am converting the days of the week to semicolons

In [ ]:
def encode(data, col, max_val):
    data[col + '_Sin'] = np.cos(2 * np.pi * data[col]/max_val)
    return data
Legal_illegal = encode(Legal_illegal, 'Week_Day', 7)
Legal_illegal = encode(Legal_illegal, 'Month', 12)
Legal_illegal

I convert the days of the month to sine and pay attention to how many days each month has. February was thought to always have 29 days

In [ ]:
List=Legal_illegal.values.tolist()
Date_Sin=[]
Month_Sin_Rules=[[1,2,3,4,5,6,7,8,9,10,11,12],[31,29,31,30,31,30,31,31,30,31,30,31]] #Κανόνες με τις μέρες που έχει ο κάθε μήνας
for i in tqdm(range(0,len(List))):
    for j in range(0,12):
        if List[i][5]==Month_Sin_Rules[0][j]:
            Max=Month_Sin_Rules[1][j]
    sin=np.sin(2 * np.pi * List[i][2]/Max)
    Date_Sin.append(sin)
Legal_illegal.insert(2, "Date_Sin", Date_Sin, True)

I delete the columns that describe the month days etc. Since now I have columns that capture these features in sine

In [ ]:
Legal_illegal=Legal_illegal.drop(['Week_Day'], axis=1)
Legal_illegal=Legal_illegal.drop(['Month'], axis=1)
Legal_illegal=Legal_illegal.drop(['Date_Of_Month'], axis=1)
Legal_illegal=Legal_illegal.drop(['Sum'], axis=1)
Legal_illegal

From the date and time I create a column that I call key. The key column describes the date and time without describing the minutes of the time. I make it in this format because hourly temperatures from the weather data are also described in this format. So I create this column to merge with the weather data later 

In [ ]:
Time=Legal_illegal['Slot_Timeint']/3600 #convert seconds to hours
Time=Time.astype(int) #i only get the hour and not the minutes
Time=Time.values.tolist()
NewT=[]
for i in range (0,len(Time)):
    Str=str(Time[i])
    if Time[i]>=10:
        NewT.append(Str)
    else:
        NewT.append('0'+Str)


Time=pd.DataFrame(NewT,columns=["Hour"])


Time= Time["Hour"].map(str)+ ':00' 
Key_Weather=Legal_illegal['Key_Date'].map(str) + ' ' + Time
Legal_illegal=Legal_illegal.drop(['Key_Date'], axis=1)
Legal_illegal.insert(1, "Key", Key_Weather, True)
Legal_illegal

I convert the columns that capture the timeslot and premarital time to be in the range 0-1 and delete the column that captures the time as I don't need it

In [ ]:
Legal_illegal['Slot_Timeint']=Legal_illegal['Slot_Timeint']/timedelta(days=1).total_seconds()
Legal_illegal['Time_Int']=Legal_illegal['Time_Int']/timedelta(days=1).total_seconds()
Legal_illegal=Legal_illegal.drop(['Year'],axis=1)

I save the DataFrame as 'Scan_Data_Reg_2.3.csv'. Where and this is the final file that will go on the notebook that I train the model on.

In [ ]:
Legal_illegal.to_csv(Project_Path+ '/Data/Scan_Data_Reg_2.3.csv')
Legal_illegal